In [1]:
import pandas as pd
from evaluate import load
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets import Dataset
from seq2seq import train_transformer, decode_with_transformer

In [2]:
data = pd.read_csv('yelp_parallel/test_en_parallel.txt', sep='\t')
data.columns = ['Style1', 'Style2']
data = data[:5000]

In [3]:
sentences_negative = data['Style1'].values.tolist()
sentences_positive = data['Style2'].values.tolist()

In [4]:
model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [5]:
instruction_transform = 'Transform negative sentiment to positive sentiment: '
inputs_transform = [f'{instruction_transform}{s}' for s in sentences_negative]
outputs_transform = sentences_positive

In [6]:
instruction_classify = 'Classify the sentiment as positive or negative: '

In [7]:
inputs_classify_neg = [f'{instruction_classify}{s}' for s in sentences_negative]
outputs_classify_neg = ['negative'] * len(sentences_negative)

In [8]:
inputs_classify_pos = [f'{instruction_classify}{s}' for s in sentences_positive]
outputs_classify_pos = ['positive'] * len(sentences_positive)

In [9]:
all_inputs = inputs_transform + inputs_classify_neg + inputs_classify_pos
all_outputs = outputs_transform + outputs_classify_neg + outputs_classify_pos

In [10]:
inputs_enc = tokenizer(all_inputs, max_length=128, truncation=True)

In [11]:
with tokenizer.as_target_tokenizer():
    outputs_enc = tokenizer(all_outputs, max_length=128, truncation=True)

C:\Users\vlade\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
train_set = Dataset.from_dict({'input_ids': inputs_enc['input_ids'],
                              'attention_mask': inputs_enc['attention_mask'],
                              'labels': outputs_enc['input_ids']})

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [14]:
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=data_collator)
optimizer = AdamW(model.parameters(), lr=0.001)
train_transformer(model, train_loader, optimizer, 3, device='cuda')

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1/3, Loss: 1.5538
Epoch 2/3, Loss: 1.1652
Epoch 3/3, Loss: 0.9277


In [15]:
predicted_sentence = decode_with_transformer(inputs_transform[0], tokenizer, model, device='cuda')
predicted_sentence

"ever since joes has changed hands it's gotten better and better."

In [16]:
reference_sentence = sentences_positive[0]
reference_sentence

"Ever since joes has changed hands it's gotten better and better."

In [17]:
bleu = load('bleu')
bleu.compute(predictions=[predicted_sentence], references=[[reference_sentence]])

{'bleu': 0.9036020036098448,
 'precisions': [0.9166666666666666,
  0.9090909090909091,
  0.9,
  0.8888888888888888],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 12,
 'reference_length': 12}

In [18]:
bertscore = load('bertscore')
bertscore.compute(predictions=[predicted_sentence], references=[reference_sentence], model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.9945324659347534],
 'recall': [0.9945324659347534],
 'f1': [0.9945324659347534],
 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.47.1)'}

In [19]:
correct = 0
total = 20

In [20]:
for i in range(10):
    pred = decode_with_transformer(inputs_classify_neg[i], tokenizer, model, device='cuda')
    if 'negative' in pred.lower():
        correct += 1

In [21]:
for i in range(10):
    pred = decode_with_transformer(inputs_classify_pos[i], tokenizer, model, device='cuda')
    if 'positive' in pred.lower():
        correct += 1

In [22]:
accuracy = correct / total
print(f'Accuracy: {accuracy}')

Accuracy: 0.9


In [23]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=data_collator)
optimizer = AdamW(model.parameters(), lr=0.0001)
train_transformer(model, train_loader, optimizer, 5, device='cuda')

Epoch 1/5, Loss: 1.8632
Epoch 2/5, Loss: 1.4576
Epoch 3/5, Loss: 1.3606
Epoch 4/5, Loss: 1.2697
Epoch 5/5, Loss: 1.1914


In [24]:
predicted_sentence = decode_with_transformer(inputs_transform[0], tokenizer, model, device='cuda')
predicted_sentence

"since joes has changed hands it's just getting better and better."

In [25]:
bleu.compute(predictions=[predicted_sentence], references=[[reference_sentence]])

{'bleu': 0.6340466277046861,
 'precisions': [0.8333333333333334,
  0.7272727272727273,
  0.6,
  0.4444444444444444],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 12,
 'reference_length': 12}

In [26]:
bertscore.compute(predictions=[predicted_sentence], references=[reference_sentence], model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.9453003406524658],
 'recall': [0.9484442472457886],
 'f1': [0.9468696713447571],
 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.47.1)'}

In [27]:
correct = 0
total = 20

In [28]:
for i in range(10):
    pred = decode_with_transformer(inputs_classify_neg[i], tokenizer, model, device='cuda')
    if 'negative' in pred.lower():
        correct += 1

In [29]:
for i in range(10):
    pred = decode_with_transformer(inputs_classify_pos[i], tokenizer, model, device='cuda')
    if 'positive' in pred.lower():
        correct += 1

In [30]:
accuracy = correct / total
print(f'Accuracy: {accuracy}')

Accuracy: 0.8


In [31]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=data_collator)
optimizer = AdamW(model.parameters(), lr=0.0005)
train_transformer(model, train_loader, optimizer, 7, device='cuda')

Epoch 1/7, Loss: 1.5613
Epoch 2/7, Loss: 1.2165
Epoch 3/7, Loss: 1.0154
Epoch 4/7, Loss: 0.8598
Epoch 5/7, Loss: 0.7261
Epoch 6/7, Loss: 0.6035
Epoch 7/7, Loss: 0.4989


In [32]:
predicted_sentence = decode_with_transformer(inputs_transform[0], tokenizer, model, device='cuda')
predicted_sentence

"Ever since joes has changed hands it's gotten better and better."

In [33]:
bleu.compute(predictions=[predicted_sentence], references=[[reference_sentence]])

{'bleu': 1.0,
 'precisions': [1.0, 1.0, 1.0, 1.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 12,
 'reference_length': 12}

In [34]:
bertscore.compute(predictions=[predicted_sentence], references=[reference_sentence], model_type='microsoft/deberta-xlarge-mnli')

{'precision': [1.0],
 'recall': [1.0],
 'f1': [1.0],
 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.47.1)'}

In [35]:
correct = 0
total = 20

In [36]:
for i in range(10):
    pred = decode_with_transformer(inputs_classify_neg[i], tokenizer, model, device='cuda')
    if 'negative' in pred.lower():
        correct += 1

In [37]:
for i in range(10):
    pred = decode_with_transformer(inputs_classify_pos[i], tokenizer, model, device='cuda')
    if 'positive' in pred.lower():
        correct += 1

In [38]:
accuracy = correct / total
print(f'Accuracy: {accuracy}')

Accuracy: 1.0


In [87]:
# Подзадача 1: 
# BLEU: 0.90, 0.63, 1.0
# BERTScore: 0.99, 0.94, 1.0
# Како заклучок од овие тестирања можеме да заклучиме дека зголемувањето на епохите директно влијае врз резултатот
# Осносно бројот на епохи има поголем удел во резултатите од ратата на учење
# Тоа се докажува со овој експеримент каде со 7 епохи и рата на учење 0.0005 добивме перфектен резултат
#
# Подзадача 2:
# Добиени резултати: 0.9, 0.8, 1.0
# Заклучоците се исти како за Задача 1
#
# Во seq2seq скриптата зголемен е опсегот од 10 на 128 зборови